# Packages

In [1]:
using FinEtools, FinEtoolsAcoustics, LinearAlgebra, StatsBase, HDF5, Random
cd(desktopPath) # change directory

# Initial setup

In [2]:
rho = 1.21 * phun("kg/m^3") # mass density
c  = 343.0 * phun("m/s") # sound speed
bulk =  c ^ 2 * rho
frequency = 500.0 # frequency of the incident wave, Hz
omega = 2 * pi * frequency
dt = 1.0 / frequency / 15
tfinal = 140 * dt
nsteps = round(tfinal / dt) + 1
println("Step takes $(sciNotation(dt, 3)) s")
println("Total of $(round(Int, nsteps)) steps")
println("Final time: $(sciNotation(tfinal, 3)) s")

Step takes 1.333E-4 s
Total of 141 steps
Final time: 1.867E-2 s


# Mesh

Geometry

In [3]:
# list of meshes
Meshes = Array{Tuple{FENodeSet, AbstractFESet}, 1}()
# Q4 discretization of large rectangular domain divisions
push!(Meshes, Q4quadrilateral([0.0 0.0; 1.2 2.6], 10, 26)) # 1
push!(Meshes, Q4quadrilateral([0.0 2.6; 1.6 5.45], 13, 30)) # 2
push!(Meshes, Q4quadrilateral([1.35 0.0; 4.05 2.6], 25, 24)) # 3
push!(Meshes, Q4quadrilateral([1.6 2.6; 4.05 5.8], 23, 34)) # 4
push!(Meshes, Q4quadrilateral([4.25 1.2; 5.25 4.5], 10, 35)) # 5
push!(Meshes, Q4quadrilateral([4.05 4.5; 5.25 5.3], 12, 9)) # 6
push!(Meshes, Q4quadrilateral([4.25 5.3; 5.25 5.85], 10, 4)) # 7
push!(Meshes, Q4quadrilateral([5.25 1.2; 7.0 3.92], 22, 29)) # 8
# merge meshes
fens, outputfes = mergenmeshes(Meshes, 0.05)
# concatenate connectivities
fes = cat(outputfes[5], cat(outputfes[6], cat(outputfes[7], outputfes[8])))
fes = cat(outputfes[1], cat(outputfes[2], cat(outputfes[3], cat(outputfes[4], fes))))
println("$(size(fens.xyz, 1)) nodes")
println("$(size(fes.conn, 1)) elements")
println("$(round(Int, size(fens.xyz, 1) * nsteps)) dataset points (using Q4)")

3362 nodes
3168 elements
474042 dataset points (using Q4)


Subdomain selection

In [4]:
# Identify boundary finite element set
bfes = meshboundary(fes)
# In case there are any unconnected nodes, remove them, and renumber the elements.
connected = findunconnnodes(fens, fes)
fens, new_numbering = compactnodes(fens, connected)
fess = renumberconn!(fes, new_numbering)
# The geometry and the solution (pressure) fields
geom = NodalField(fens.xyz)
P = NodalField(zeros(FCplxFlt, size(fens.xyz, 1), 1));

Export domain

In [5]:
# Export three VTK files: one for the interior of the fluid, and one for the boundary
# vtkexportmesh("./vtkFiles/interior.vtk", fes.conn, fens.xyz, FinEtools.MeshExportModule.VTK.Q4);

# Discrete model

Setup

In [6]:
# Number the degrees of freedom in the pressure field.
numberdofs!(P)
# Create the finite element machine for the fluid.
material = MatAcoustFluid(bulk, rho)
femm  =  FEMMAcoust(IntegDomain(fes, GaussRule(3, 2)), material)
# Use the machine calculate the acoustic stiffness and mass matrices.
S  =  acousticstiffness(femm, geom, P)
C  =  acousticmass(femm, geom, P)
# "damping" matrix for the absorbing boundary conditions (ABCs)
D  =  acousticABC(
  FEMMAcoustSurf(IntegDomain(bfes, GaussRule(2, 2)), material), geom, P
)
# node IDs for load positions
loadNode = eachindex(fens) |> collect
# initial condition as load in nodes of an element
function pointLoad(dpdn, xyz, J, label, t)::Float64
  dpdn[1] = t == dt ? -rho * 0.02 : 0.0
end;

Point selection

In [7]:
ICamount = 1000 # amound of initial conditions
# IDs of load elements (different initial conditions)
loadID = (collect(1 : size(fes.conn, 1)) |> shuffle)[1 : ICamount]
@show size(loadID)
# sensors are chosen as the first node of each load element.
# therefore: number of sensors = number of ICs

# random points in random instants.
# points y for evaluating solution G(u)(y).
# inputs to trunk network.
# matrix with rows [nodeID x y t]
yAmount = 1000
# IDs of chosen nodes
yNodeID = (collect(1 : size(fens.xyz, 1)) |> shuffle)[1 : yAmount]
y = hcat(
  yNodeID,
  fens.xyz[yNodeID, :],
  rand(1:nsteps, yAmount)' |> vec
)
@show size(y)
println("HDF5 file contains $(
  4 * yAmount + ICamount * yAmount
) values across $(ICamount * yAmount) samples.")

size(loadID) = (1000,)


size(y) = (1000, 4)
HDF5 file contains 1004000 values across 1000000 samples.


Solution

In [8]:
# 
data = Dict(
  # spatial-temporal points for evaluating G(u). trunk inputs
  "solEvalPoints" => y,
  # G(u)(y). Targets for DeepONet outputs.
  # each row refers to an IC case; each column, to a point y
  "solutionValues" => zeros(Float32, (ICamount, yAmount))
)
for (count, loadElementID) in enumerate(loadID)
  evalPoints = 0
  count % 100 == 0 && println("Element $count/$ICamount")
  # element to be loaded in current IC
  dipfemm = FEMMAcoustSurf(IntegDomain(subset(fes, [loadElementID]), GaussRule(2, 2)), material)
  t = 0.0 # Initial time
  # Solve the transient acoustics equations.
  # The loop executes inside the "let" local scope
  P1 = let
    P0 = deepcopy(P)
    P0.values .= 0.0 # initially all pressure is zero
    vP0 = gathersysvec(P0)
    vQ0 = zeros(eltype(vP0), size(vP0))
    # The `P1` field will be the output of this computation:
    # the final value of the pressure field
    P1 = deepcopy(P0)
    fi = ForceIntensity( # Initial load
      FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
    )
    La0 = distribloads(dipfemm, geom, P1, fi, 2)
    A = (2.0 / dt) * S + D + (dt / 2.0) * C
    step = 0
    while t <= tfinal
      step += 1
      t += dt
      # Update load
      fi = ForceIntensity( # Initial load
        FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
      )
      La1 = distribloads(dipfemm, geom, P1, fi, 2)
      # Solve for the rate of the pressure
      vQ1 = A \ ((2 / dt) * (S * vQ0) - D * vQ0 - C * (2 * vP0 + (dt / 2) * vQ0) + La0 + La1)
      # Update the value of the pressure
      vP1 = vP0 + (dt / 2) * (vQ0 + vQ1)
      # Swap variables for next step  
      vP0 = deepcopy(vP1); vQ0 = deepcopy(vQ1)
      P1 = scattersysvec!(P1, vec(vP1))
      P0 = deepcopy(P1); La0 = deepcopy(La1)
      # find rows in data["solEvalPoints"] that refer to current timestep
      yRows = findall(k -> k[4] == step, eachrow(data["solEvalPoints"]))
      evalPoints += length(yRows)
      # read pressure values in nodes of interest in current timestep, and
      # use those values to populate the row of data["solutionValues"]
      # referring to the current IC case
      data["solutionValues"][count, yRows] .= (real.(P1.values) |> vec)[Int.(data["solEvalPoints"][yRows, 1])]
    end
    P1
  end
end

Element 100/1000


Element 200/1000


Element 300/1000


Element 400/1000


Element 500/1000


Element 600/1000


Element 700/1000


Element 800/1000


Element 900/1000


Element 1000/1000

# Data

Normalize

In [9]:
standardData = copy(data)
unitData = copy(data)
# standardize
standardData["solEvalPoints"] = Float32.(copy(data["solEvalPoints"]))
standardData["solEvalPoints"][:, 2:4] = standardize(ZScoreTransform, data["solEvalPoints"][:, 2:4], dims = 1)
standardData["solutionValues"] = Float32.(standardize(ZScoreTransform, data["solutionValues"]))
# unit normalization
unitData["solEvalPoints"] = Float32.(copy(data["solEvalPoints"]))
unitData["solEvalPoints"][:, 2:4] = standardize(UnitRangeTransform, data["solEvalPoints"][:, 2:4], dims = 1)
unitData["solutionValues"] = Float32.(standardize(UnitRangeTransform, data["solutionValues"]));

Save in HDF5

In [10]:
# create files for versions of dataset: raw, standardized, and unit normalized
for (source, name) in [(data, "data") (standardData, "standardData") (unitData, "unitData")]
  h5open("./vtkFiles/$name.h5", "w") do dataFile
    for (key, value) in source
      dataFile[key] = value
    end
  end
end;

Summary statistics of file

In [2]:
for name in ["data" "standardData" "unitData"]
  println("########      $name      ########")
  h5open("./vtkFiles/$name.h5", "r") do f
    for field in HDF5.get_datasets(f)
      println("\n", HDF5.name(field)[2 : end])
      d = HDF5.read(field)
      @show size(d)
      statsum(d)
      println(d[1, :])
    end
  end
  println()
end;

########      data      ########



solEvalPoints


size(d) = (1000, 4)


Summary Stats:
Length:         4000
Missing Count:  0
Mean:           432.918457
Minimum:        0.000000
1st Quartile:   2.984783
Median:         6.681818
3rd Quartile:   133.000000
Maximum:        3355.000000
Standard deviation: 8.604E2
3974/4000 (99.4%) non-zero elements./n


[2348.0, 4.95, 2.8028571428571425, 24.0]

solutionValues
size(d) = (1000, 1000)


...

0.5071488, 0.326581]

########      unitData      ########

solEvalPoints
size(d) = (1000, 4)
Summary Stats:
Length:         4000
Missing Count:  0
Mean:           414.599640
Minimum:        0.000000
1st Quartile:   0.333392
Median:         0.628571
3rd Quartile:   1.250000
Maximum:        3355.000000
Standard deviation: 8.6847E2
3970/4000 (99.2%) non-zero elements./n
Float32[2348.0, 0.70714283, 0.47912088, 0.16428572]

solutionValues
size(d) = (1000, 1000)


Summary Stats:
Length:         1000000
Missing Count:  0
Mean:           0.823737
Minimum:        0.000000
1st Quartile:   0.785687
Median:         0.837336
3rd Quartile:   0.885045
Maximum:        1.000000
Standard deviation: 9.8351E-2
998998/1000000 (99.9%) non-zero elements./n
Float32[0.9919536, 0.74402666, 0.88283515, 0.84309363, 0.8828326, 0.80422133, 0.8866377, 0.88283515, 0.8440204, 0.8375017, 0.88283515, 0.88283515, 0.88283515, 0.88283515, 0.88283515, 0.88283515, 0.8603265, 0.88283515, 0.81019634, 0.8474363, 0.84127486, 0.8828342, 0.8828272, 0.88283515, 0.71338093, 0.8548288, 0.8479888, 0.8828417, 0.77746207, 0.8828348, 0.7522927, 0.8828052, 0.88283515, 0.7988845, 0.88283515, 0.88283515, 0.7903634, 0.88283515, 0.8442718, 0.8805276, 0.8693917, 0.77112055, 0.8944007, 0.88283515, 0.865213, 0.8828289, 0.8675456, 0.84353954, 0.63857013, 0.871122, 0.8365279, 0.88283515, 0.88283056, 0.88283515, 0.8828357, 0.88283515, 0.88283515, 0.88283515, 0.80508983, 0.8142815, 0.88283515, 0.773374

 0.88297963, 0.8703174, 0.88283515, 0.8597038, 0.8716004, 0.8944666, 0.88283515, 0.8506248, 0.84300077, 0.8614048, 0.81417406, 0.88283515, 0.88283515, 0.88283515, 0.88283515, 0.8828345, 0.8828354, 0.88291335, 0.88283515, 0.88283515, 0.88283515, 0.798276, 0.8536624, 0.41290775, 0.8619707, 0.87968004, 0.7669711, 0.80995166, 0.88283515, 0.7957316, 0.8047823, 0.78957367, 0.8555467, 0.88283515, 0.88283515, 0.88283515, 0.88283515, 0.88283515, 0.85718465, 0.7892238, 0.739253, 0.8828356, 0.88283515, 0.88283515, 0.84704983, 0.84300816, 0.88283515, 0.8828369, 0.8719953, 0.88283515, 0.81125915, 0.84153557, 0.88283515, 0.7711932, 0.88283515, 0.88283515, 0.88283515, 0.8828347, 0.8828428, 0.88283515, 0.882835, 0.8827702, 0.88283515, 0.8556071, 0.83718336, 0.8812939, 0.8854791, 0.82428926, 0.8122379, 0.88283235, 0.88283503, 0.88283515, 0.86290056, 0.8537105, 0.88283515, 0.7945067, 0.85753864, 0.85241234, 0.70194966, 0.882836, 0.88283515, 0.8802668, 0.78096455, 0.8828342, 0.88283515, 0.88283515, 0.882

 0.8836902, 0.85001075, 0.88283515, 0.88283515, 0.88283515, 0.848811, 0.86363274, 0.88283515, 0.88283515, 0.88283515, 0.73678285, 0.88283515, 0.78942823, 0.88283515, 0.7963847, 0.82103664, 0.8149714, 0.88283515, 0.8828348, 0.88283515, 0.90970623, 0.88283515, 0.88283515, 0.88283813, 0.8387352, 0.85984164, 0.8828357, 0.88283515, 0.807315, 0.77423996, 0.88283515, 0.88283515, 0.88283515, 0.80050564, 0.7858973, 0.88283515, 1.0, 0.9731913, 0.74966806, 0.88283515, 0.7547448, 0.8743643, 0.8452263, 0.856008, 0.80343324, 0.8293448, 0.72869235, 0.8016962, 0.84995455, 0.88283515, 0.85711235, 0.79543227, 0.88283515, 0.88283515, 0.56173223, 0.8826228, 0.7932521, 0.8427467, 0.88141376, 0.88283515, 0.88283515, 0.8508461, 0.88283515, 0.854665, 0.80170083, 0.7636715, 0.88283515, 0.88283515, 0.88283515, 0.8828354, 0.8034584, 0.8644857, 0.885673, 0.88283515, 0.88283515, 0.9085916, 0.8028008, 0.8645844, 0.45160592, 0.852944, 0.66714054, 0.80698276, 0.88283515, 0.791554, 0.8695607, 0.88283515, 0.88283515, 0

0.80124134, 0.88513064, 0.7566131, 0.88283515, 0.88283515, 0.87142926, 0.8402832, 0.88283426, 0.88283515, 0.85113406, 0.88283515, 0.88283515, 0.8794363, 0.88283515, 0.88283515, 0.88283515, 0.8467033, 0.88283515, 0.88283515, 0.88283515, 0.89711225, 0.8048762, 0.77429175, 0.76757556, 0.88283515, 0.7870643, 0.83567345, 0.8377634, 0.79932576, 0.88283515, 0.84514076, 0.88283515, 0.787208, 0.88283515, 0.88283515, 0.88283515, 0.84470254, 0.8080562, 0.88283515, 0.84111965, 0.84927726, 0.88283515, 0.8828316, 0.76564604, 0.84496677, 0.88283515, 0.88251686, 0.88283515, 0.88283515, 0.88283515, 0.88283515, 0.8689136, 0.88283515, 0.8828351, 0.8093631, 0.8851347, 0.8648873, 0.8157542, 0.88283515, 0.88283515, 0.8828409, 0.8828343, 0.79620624, 0.88196176, 0.8532854, 0.88283515, 0.8446887, 0.88283515, 0.8545433, 0.8537017, 0.88283515, 0.79524434, 0.88283515, 0.87031657]

